In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import pandas as pd 
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
from sklearn import datasets, linear_model
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV


In [2]:
flows = pd.read_excel("Flows_Clean_Small.xlsx", index= False)
returns = pd.read_excel("Returns_Clean_Small.xlsx", index= False)

In [3]:
flows.head()

,Dates,25320,30402,76082,18112,58026,43350,52484,29612,68340,...,21936,76745,42438,19692,77465,71116,38295,76804,38093,70965
0,19930401,0.309825,0.000000,-1.516741,3.627825,-14.838664,1.774343,-6.641067,0.241909,-2.740650,...,2.920712,-33.876369,-2.606596,2.208227,5.497335,-0.128825,1.383739,-5.068082,-1.789447,0.677818
1,19930402,1.644644,-2.529088,0.918029,0.897479,-8.030461,0.714675,-1.976790,-0.604774,1.616935,...,-8.372823,-3.867574,-1.537929,8.144802,-4.737128,-0.161226,0.371736,-1.665555,-1.625130,2.131676
2,19930405,-0.601645,1.170176,2.243447,-2.621316,0.502112,1.453849,11.404063,1.504184,2.434260,...,-4.259489,0.444482,13.484679,-2.214915,20.221963,0.342782,0.441380,3.706494,-0.657694,-0.247377
3,19930406,-2.290313,0.531704,0.326943,-6.457085,-12.055085,4.059845,-25.944941,-0.599611,1.164004,...,-1.573684,7.096013,-8.120186,-2.956802,-7.055059,0.423770,-0.715924,2.113024,0.030046,0.056158
4,19930407,-2.389404,0.558868,9.608559,-17.121311,3.656666,1.655169,5.877173,-1.080171,2.120861,...,0.257004,-29.953501,0.776082,6.062879,-2.351458,0.253678,-0.484014,5.143622,8.787357,-0.052975


In [4]:
returns.head()

,date,25320,30402,76082,18112,58026,43350,52484,29612,68340,...,21936,76745,42438,19692,77465,71116,38295,76804,38093,70965
0,19930401,-0.005634,-0.009709,0.015957,0.016461,-0.006135,-0.002755,-0.011236,-0.010753,0.007874,...,-0.018072,-0.022059,-0.007042,0.007299,0.000000,0.000000,0.125000,0.000000,-0.014085,0.000000
1,19930402,-0.005666,0.000000,0.000000,-0.016194,-0.024691,-0.011050,-0.011364,0.005435,-0.054688,...,-0.036810,0.007519,-0.021277,-0.021739,-0.054688,-0.004202,-0.111111,-0.026087,0.000000,-0.013889
2,19930405,-0.025788,-0.019608,0.005236,-0.004115,-0.012658,0.033520,-0.068966,0.005405,0.000000,...,0.002123,-0.007463,0.014493,0.011111,0.049587,-0.004219,0.125000,0.024554,-0.003571,0.000000
3,19930406,-0.017647,-0.010000,0.000000,0.000000,0.006410,-0.005405,-0.061728,0.021505,-0.008264,...,-0.023305,0.000000,0.000000,-0.080586,-0.015748,-0.004237,-0.111111,-0.006536,0.000000,-0.014085
4,19930407,-0.056886,0.010101,0.005208,-0.012397,0.012739,0.029891,0.026316,0.000000,-0.016667,...,0.015184,-0.015038,0.000000,0.019920,0.000000,-0.004255,0.000000,0.015351,0.017921,0.014286


### X_full

In [7]:

def X_full_(stock_number, stock_lag, cross_stock_lag,returns,flows):
    
    
    stocks = [s for s in list(returns)[1:] if s != stock_number]
    
    colnames_full = []
    colnames_small = ['rt-'+ str(i+1) for i in range(stock_lag)] + ['ft-'+ str(i+1) for i in range(stock_lag)]
    
    for j in range(cross_stock_lag):
        colnames_full +=  [str(stocks[i]) + 'ft-' + str(j+1) for i in range(len(stocks))]+[str(stocks[i]) + 'rt-' + str(j+1) for i in range(len(stocks))]
                    
    X_full = pd.DataFrame(columns = ['rt'] + colnames_small + colnames_full)            
    
    for i in range(stock_lag,len(flows)):
        tmp_return_small = [returns[stock_number][i-j-1] for j in range(stock_lag)]
        tmp_flow_small = [flows[stock_number][i-j-1] for j in range(stock_lag)]
        
        tmp_return_full = []
        tmp_flow_full = []

        for k in range(len(stocks)):
            tmp_return_full += [returns[stocks[k]][i-j-1] for j in range(cross_stock_lag)]
            tmp_flow_full += [flows[stocks[k]][i-j-1] for j in range(cross_stock_lag)]
    
        X_full.loc[i] = [returns[stock_number][i]] + tmp_return_small + tmp_flow_small + tmp_flow_full + tmp_return_full
        X_full = X_full.reset_index(drop=True)
        
        
    return X_full
                   
                   

### Random Forest Regressor

- Choose window size
- In Each window: Perform Time Series Nested-Cross-Validation to find optimal parameters for random forest regressor
- Fit on whole window with Lasso(alpha = optimal)
- Predict next return
- Roll to nex window
- Do this for every window
- Choose window that performs best

In [8]:
windows = [90,120,150,180]
error = [0] * len(windows)

optimal_window = 0
optimal_windows = []


tscv = TimeSeriesSplit(5)

stocks = [25320, 58026, 11622, 62260, 77371, 25080, 10908, 44821, 63159, 23819, 75382, 10302, 61138, 45671, 39773, 75072, 85914, 58800, 77465, 38295]

rfc = RandomForestRegressor(random_state=42)

param_grid = { 
    'n_estimators': [50],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8,10],
    "min_samples_split": [5],
    "min_samples_leaf": [30],
    "max_leaf_nodes": [25],
    "min_weight_fraction_leaf": [0.1]
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv = tscv, n_jobs = -1)

for stock in stocks:
     
    X_full = X_full_(stock, 3, 1,returns,flows)
    X_train = X_full.iloc[0:1501,]
    tmp_min_error = 1000
    
    count = 0
   
    for window in windows:
        
        y_pred = []
        

        for i in range(len(X_train)- window):
            X = X_train.iloc[i:i+window, X_train.columns != 'rt'].values
            y = X_train['rt'].iloc[i:i+window].values
            CV_rfc.fit(X,y)
            
            rfr = RandomForestRegressor(n_estimators=CV_rfc.best_params_['n_estimators'], max_depth = CV_rfc.best_params_['max_depth'], 
                             min_samples_split=5, min_samples_leaf=30, min_weight_fraction_leaf=0.1, 
                             max_features=CV_rfc.best_params_['max_features'], max_leaf_nodes=25)
            
            rfr.fit(X,y)
            
            y_pred.append(float(rfr.predict([list(X_train.iloc[i+window, X_train.columns != 'rt'])])))
        print("Currently in window ", window)
            

        #If this window size performed better than the previous one: save it. Do nothing otherwise.
        error[count] += mean_squared_error(X_train['rt'].iloc[window:], y_pred)
        count = count + 1
        print("error is ", error)
        if mean_squared_error(X_train['rt'].iloc[window:], y_pred) < tmp_min_error:
            optimal_window = window
            tmp_min_error = mean_squared_error(X_train['rt'].iloc[window:], y_pred)
    
    optimal_windows.append(optimal_window)
    print('Optimal window for stock ' + str(stock) + ': ' + str(optimal_window))      



Currently in window  90
error is  [0.0002598001409299434, 0, 0, 0]
Currently in window  120
error is  [0.0002598001409299434, 0.00026106595121689306, 0, 0]
Currently in window  150
error is  [0.0002598001409299434, 0.00026106595121689306, 0.00026036622929464435, 0]
Currently in window  180
error is  [0.0002598001409299434, 0.00026106595121689306, 0.00026036622929464435, 0.000262282438927752]
Optimal window for stock 25320: 90
Currently in window  90
error is  [0.0008608115168285274, 0.00026106595121689306, 0.00026036622929464435, 0.000262282438927752]
Currently in window  120
error is  [0.0008608115168285274, 0.0008632582272379121, 0.00026036622929464435, 0.000262282438927752]
Currently in window  150
error is  [0.0008608115168285274, 0.0008632582272379121, 0.0008623179655215395, 0.000262282438927752]
Currently in window  180
error is  [0.0008608115168285274, 0.0008632582272379121, 0.0008623179655215395, 0.000870400464126676]
Optimal window for stock 58026: 90
Currently in window  90
e

Currently in window  180
error is  [0.021165444222104583, 0.020907734215736346, 0.020057896149845017, 0.019493148979117553]
Optimal window for stock 75072: 90
Currently in window  90
error is  [0.022438749668684064, 0.020907734215736346, 0.020057896149845017, 0.019493148979117553]
Currently in window  120
error is  [0.022438749668684064, 0.022174990609612977, 0.020057896149845017, 0.019493148979117553]
Currently in window  150
error is  [0.022438749668684064, 0.022174990609612977, 0.021332378820862155, 0.019493148979117553]
Currently in window  180
error is  [0.022438749668684064, 0.022174990609612977, 0.021332378820862155, 0.020752716901535566]
Optimal window for stock 85914: 180
Currently in window  90
error is  [0.023792379487135393, 0.022174990609612977, 0.021332378820862155, 0.020752716901535566]
Currently in window  120
error is  [0.023792379487135393, 0.02354009452558201, 0.021332378820862155, 0.020752716901535566]
Currently in window  150
error is  [0.023792379487135393, 0.0235

### Moving Average 

In [9]:
def moving_average(series, n):
    """
        Calculate average of last n observations
    """
    return np.average(series[-n:])


Currently in window  90
error is  [8.339602855730453, 0, 0, 0]
Currently in window  120
error is  [8.339602855730453, 8.414091135537607, 0, 0]
Currently in window  150
error is  [8.339602855730453, 8.414091135537607, 8.315960290785252, 0]
Currently in window  180
error is  [8.339602855730453, 8.414091135537607, 8.315960290785252, 8.394386292533484]
Optimal window for stock index: 150

In [10]:
def plotMovingAverage(series, window):

    """
        series - dataframe with timeseries
        window - rolling window size 
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 
    """
    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

        
    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

In [14]:
def plotDRMSE(series_pred, series_true, window):
    rolling_mean = list(series_true.rolling(window = window).mean())[window:]
    dRMSE = []
    rmse_mean = []
    rmse_model = []
    
    for i in range(len(rolling_mean)):
        rmse_mean.append(sqrt(mean_squared_error(rolling_mean[:i+1], series_true[window:window+i+1])))
        rmse_model.append(sqrt(mean_squared_error(series_pred[:i+1], series_true[window:window+i+1])))
                         
        dRMSE.append(rmse_mean[i]-rmse_model[i])
    
    plt.plot(range(len(dRMSE)), dRMSE, label='$\delta RMSE$')

In [ ]:
all_stocks_pred = []

In [17]:
window = 220
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv = tscv, n_jobs = -1)
stocks = stocks = [25320, 58026, 11622, 62260, 77371, 25080, 10908, 44821, 63159, 23819, 75382, 10302, 61138, 45671, 39773, 75072, 85914, 58800, 77465, 38295]

for idx, stock in enumerate(stocks):
    
    
    X_test = X_full_(stock, 3, 1,returns,flows).iloc[1501:,] 
    y_pred_test = []

    for i in range(len(X_test)-window):
            X = X_test.iloc[i:i+window, X_test.columns != 'rt'].values
            y = X_test['rt'].iloc[i:i+window].values
            CV_rfc.fit(X,y)

            rfr = RandomForestRegressor(n_estimators=CV_rfc.best_params_['n_estimators'], max_depth = CV_rfc.best_params_['max_depth'], 
                            min_samples_split=5, min_samples_leaf=30, min_weight_fraction_leaf=0.1, 
                            max_features=CV_rfc.best_params_['max_features'], max_leaf_nodes=25)

            rfr.fit(X,y)
            y_pred_test.append(float(rfr.predict([list(X_test.iloc[i+window, X_train.columns != 'rt'])])))

            if(i % 50 == 0):
                print(str((i/(len(X_test)-window))*100) + "% done")
    all_stocks_pred.append(y_pred_test)
    print(str(stock)+ " done")

0.0% done
21.367521367521366% done
42.73504273504273% done
64.1025641025641% done
85.47008547008546% done
25320 done
0.0% done
21.367521367521366% done
42.73504273504273% done
64.1025641025641% done
85.47008547008546% done
58026 done
0.0% done
21.367521367521366% done
42.73504273504273% done
64.1025641025641% done
85.47008547008546% done
11622 done
0.0% done
21.367521367521366% done
42.73504273504273% done
64.1025641025641% done
85.47008547008546% done
62260 done
0.0% done
21.367521367521366% done
42.73504273504273% done
64.1025641025641% done
85.47008547008546% done
77371 done
0.0% done
21.367521367521366% done
42.73504273504273% done
64.1025641025641% done
85.47008547008546% done
25080 done
0.0% done
21.367521367521366% done
42.73504273504273% done
64.1025641025641% done
85.47008547008546% done
10908 done
0.0% done
21.367521367521366% done
42.73504273504273% done
64.1025641025641% done
85.47008547008546% done
44821 done
0.0% done
21.367521367521366% done
42.73504273504273% done
64.10

In [28]:
final = all_stocks_pred

In [29]:
df = pd.DataFrame(final)

In [30]:
df

,0,1,2,3,4,5,6,7,8,9,...,224,225,226,227,228,229,230,231,232,233
0,0.000479,-0.000924,-0.005655,-0.003156,-0.009724,-0.000467,-0.003510,0.000338,-0.007405,-0.000805,...,0.000955,0.005166,0.001609,0.000262,0.004289,0.000832,0.002035,0.000841,0.001033,0.001534
1,-0.011152,-0.002187,-0.013640,-0.008967,0.001656,0.005273,-0.010924,-0.010458,-0.001111,-0.000062,...,-0.011969,0.003103,0.001918,0.001629,0.000742,-0.006233,-0.010823,-0.000928,-0.002698,-0.002495
2,0.006558,-0.018999,0.008181,-0.015483,0.004213,0.007204,-0.015585,0.004318,-0.010816,0.005856,...,0.012209,0.007408,0.003626,0.003240,0.005210,-0.014503,0.000438,-0.016397,0.003824,0.005914
3,0.003870,0.001163,0.004670,0.004272,-0.002493,-0.002752,0.002467,0.000467,0.007343,-0.002667,...,-0.000358,0.001420,-0.001556,-0.000031,-0.004820,-0.002736,0.004693,0.005018,0.007124,0.007718
4,-0.001878,-0.001272,-0.002718,0.000303,-0.004637,-0.002255,-0.003038,-0.001990,0.003196,-0.001846,...,0.001677,0.005161,0.004201,0.002666,0.006762,0.007414,0.010065,-0.001132,0.002385,0.001321
5,-0.000965,-0.000458,-0.000106,-0.000959,-0.000716,-0.000699,-0.001091,-0.000443,-0.000858,-0.001227,...,0.000393,-0.001634,-0.001548,-0.000038,0.000078,-0.000351,-0.001212,-0.000374,0.002719,-0.000824
6,-0.006402,0.012247,0.002187,0.003349,-0.006897,0.004485,0.004752,-0.013069,0.006037,-0.002589,...,0.002272,0.002220,0.000529,-0.000215,-0.002337,0.002153,0.010314,0.003703,0.014135,-0.002716
7,0.008745,0.006824,0.001179,0.003838,0.004175,0.003079,0.003080,0.006308,0.001842,0.003900,...,0.006928,0.007045,0.003935,0.013278,0.001764,-0.001335,0.009529,0.000757,0.009243,0.011071
8,-0.001132,0.000051,-0.004406,-0.001053,0.001468,-0.000024,-0.000283,0.002982,-0.001618,-0.002173,...,0.003499,-0.001251,-0.006726,-0.001910,-0.001070,-0.009337,-0.006702,-0.001605,-0.000478,0.001812
9,-0.001568,-0.000562,0.002346,-0.001766,0.001637,0.000433,-0.001463,0.004877,0.003511,-0.002265,...,-0.001317,0.002079,-0.000551,0.001751,0.001878,0.001315,0.000889,0.002674,0.004488,0.000987


In [31]:
avg_error = df.sum(axis=1)/234

In [32]:
avg_error

0    -0.000697
1    -0.004064
2    -0.000519
3    -0.001369
4    -0.000152
5    -0.001652
6     0.003931
7     0.006305
8    -0.001401
9     0.001065
10    0.001826
11    0.005244
12    0.002632
13    0.005542
14    0.000687
15    0.002612
16    0.001774
17   -0.002088
18   -0.001923
19   -0.002095
dtype: float64